In [11]:
#convert annotations.xml to xlsx file
import xml.etree.ElementTree as ET
import pandas as pd

# Load XML file
tree = ET.parse('/task_emotion_annotations_vggface2/annotations.xml')
root = tree.getroot()

# Create lists to store extracted data
image_data = []

# Loop through each image element
for image in root.findall('image'):
    image_id = int(image.get('id'))
    image_name = image.get('name')
    bbox = image.find('box')
    if bbox is not None:
        xtl = float(bbox.get('xtl'))
        ytl = float(bbox.get('ytl'))
        xbr = float(bbox.get('xbr'))
        ybr = float(bbox.get('ybr'))
        dominance = int(bbox.find("attribute[@name='Dominance']").text)
        arousal = int(bbox.find("attribute[@name='Arousal']").text)
        valence = int(bbox.find("attribute[@name='Valence']").text)
        emotion_label = bbox.find("attribute[@name='prevalent emotion']").text

        image_data.append([
            image_id,
            image_name,
            xtl, ytl, xbr, ybr,
            dominance,
            arousal,
            valence,
            emotion_label
        ])

# Create a DataFrame
df = pd.DataFrame(image_data, columns=[
    'Image ID',
    'Image Name',
    'xtl', 'ytl', 'xbr', 'ybr',
    'Dominance',
    'Arousal',
    'Valence',
    'Emotion Label'
])

# Save the DataFrame to an Excel file
df.to_excel('annotations.xlsx', index=False)


In [12]:
#convert xlsx to txt file for inference
import pandas as pd

# Read the Excel file
df = pd.read_excel('/task_emotion_annotations_vggface2/annotations.xlsx')

# Format data and write to a text file
with open('final_inference_input.txt', 'w') as txt_file:
    for index, row in df.iterrows():
        image_name = "/task_emotion_annotations_vggface2/default/Person/" + row['Image Name']
        xtl = int(round(row['xtl']))
        ytl = int(round(row['ytl']))
        xbr = int(round(row['xbr']))
        ybr = int(round(row['ybr']))
        emotion_label = row['Emotion Label']
        valence = row['Valence']  # Add Valence
        arousal = row['Arousal']  # Add Arousal
        dominance = row['Dominance']  # Add Dominance
        
        line = f"{image_name} {xtl} {ytl} {xbr} {ybr} {emotion_label} {valence} {arousal} {dominance}\n"  # Include Valence, Arousal, and Dominance
        txt_file.write(line)

print("Text file created successfully.")


Text file created successfully.


In [16]:
import pandas as pd

# Read the text file
data = []
with open('/task_emotion_annotations_vggface2/de_exp/results/inference_list_emotic_res50_colab.txt', 'r') as txt_file:
    for line in txt_file:
        parts = line.strip().split()
        image_name = parts[0].replace("/task_emotion_annotations_vggface2/default/", "")
        emotion_label = ' '.join(parts[1:-3])
        valence = float(parts[-3])
        arousal = float(parts[-2])
        dominance = float(parts[-1])
        data.append([image_name, emotion_label, valence, arousal, dominance])

# Create a DataFrame and save as Excel
df = pd.DataFrame(data, columns=['Image Name', 'Emotion Label', 'Valence', 'Arousal', 'Dominance'])
df.to_excel('inference_output_emotic_res50_colab_2.xlsx', index=False)

print("Excel file created successfully.")

Excel file created successfully.


In [19]:
#used to create images with bounding boxes and labels from inference output
import pandas as pd
import cv2
import os

# Read the Excel file
excel_path = '/task_emotion_annotations_vggface2/bbox and both labels_emotic_res50_colab.xlsx'
output_dir = 'output_images_emotic_res50_colab'
os.makedirs(output_dir, exist_ok=True)

df = pd.read_excel(excel_path)

for index, row in df.iterrows():
    image_name = row['Image Name']
    image_path = os.path.join('/task_emotion_annotations_vggface2/default/Person', image_name)
    image = cv2.imread(image_path)
    
    xtl, ytl, xbr, ybr = int(row['xtl']), int(row['ytl']), int(row['xbr']), int(row['ybr'])
    cv2.rectangle(image, (xtl, ytl), (xbr, ybr), (0, 0, 255), 2)
    
    label = f"GT: {row['Ground Truth Label']}  Pred: {row['Predicted Label']}"
    cv2.putText(image, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    output_image_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_image_path, image)

print("Images with bounding boxes and labels created successfully.")


Images with bounding boxes and labels created successfully.
